In [20]:
# 1. Import Plotly
import plotly.graph_objects as go
import pandas as pd
import glob
from pathlib import Path

In [21]:
# find relevant data files
csv_files = glob.glob("../logs/sensor_data_*.csv")
print(f"found {len(csv_files)} files: ")
for file in csv_files:
    print(f" - {file}")

# load the most recent file
latest_file = max(csv_files, key=lambda f: Path(f).stat().st_mtime)
df = pd.read_csv(latest_file)
#df = pd.read_csv("scripts/sensor_data_20251104_102135.csv")
print(f"Loaded {len(df)} data points")

found 3 files: 
 - ../logs/sensor_data_20251113_220554.csv
 - ../logs/sensor_data_20251113_232918.csv
 - ../logs/sensor_data_20251113_234100.csv
Loaded 700 data points


In [22]:
# Normalize column names to handle both old and new CSV formats
# New format (from bin2csv): accel_x_g, gyro_x_dps, mag_x_gauss, timestamp_sec
# Old format: ax_g, gx_dps, mx_raw_gauss, timestamp_ms

if 'accel_x_g' in df.columns:
    # New format - rename to old format for compatibility
    df.rename(columns={
        'accel_x_g': 'ax_g',
        'accel_y_g': 'ay_g', 
        'accel_z_g': 'az_g',
        'gyro_x_dps': 'gx_dps',
        'gyro_y_dps': 'gy_dps',
        'gyro_z_dps': 'gz_dps',
        'mag_x_gauss': 'mx_raw_gauss',
        'mag_y_gauss': 'my_raw_gauss',
        'mag_z_gauss': 'mz_raw_gauss'
    }, inplace=True)
    
    # Convert timestamp_sec to timestamp_ms if needed
    if 'timestamp_sec' in df.columns and 'timestamp_ms' not in df.columns:
        df['timestamp_ms'] = df['timestamp_sec'] * 1000.0
    
    print("✓ Converted new CSV format to legacy column names")
elif 'ax_g' in df.columns:
    print("✓ Using legacy CSV format")
else:
    print("⚠️  WARNING: Unknown CSV format!")
    print("Available columns:", df.columns.tolist())

✓ Using legacy CSV format


In [23]:
# check data structure
print("data shape:", df.shape);
print("\nColumns:", df.columns.tolist())
print("\nFirst few rows:")
df.head()

data shape: (700, 10)

Columns: ['timestamp_ms', 'ax_g', 'ay_g', 'az_g', 'gx_dps', 'gy_dps', 'gz_dps', 'mx_raw_gauss', 'my_raw_gauss', 'mz_raw_gauss']

First few rows:


,timestamp_ms,ax_g,ay_g,az_g,gx_dps,gy_dps,gz_dps,mx_raw_gauss,my_raw_gauss,mz_raw_gauss
0,0,-0.090463,0.005490,-1.01620,0.02625,-1.76750,-5.06625,0.22616,-0.12088,-0.14600
1,203,-0.090890,0.006344,-1.01620,-0.08750,-1.68875,-4.93500,0.22600,-0.12312,-0.14896
2,407,-0.090097,0.005673,-1.02083,0.15750,-1.12875,-4.52375,0.22416,-0.12040,-0.14528
3,611,-0.095526,0.003233,-1.01248,0.00875,-1.63625,-5.08375,0.22592,-0.12176,-0.14800
4,815,-0.095160,0.002806,-1.01138,0.11375,-1.93375,-4.69875,0.22592,-0.12048,-0.14664


In [24]:
# Create time column
df['time_sec'] = (df['timestamp_ms'] - df['timestamp_ms'].iloc[0]) / 1000.0

# Simple 2D line plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['mx_raw_gauss'], name='Mag X'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['my_raw_gauss'], name='Mag Y'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['mz_raw_gauss'], name='Mag Z'))
fig.update_layout(title='Magnetometer Data', xaxis_title='Time (s)', yaxis_title='Gauss')
fig.show()

## Magnetometer Data Variation Analysis

Check if the magnetometer data has sufficient 3D variation for ellipsoid fitting.

In [25]:
import numpy as np

# Extract magnetometer data
x = df['mx_raw_gauss']
y = df['my_raw_gauss']
z = df['mz_raw_gauss']

print("="*60)
print("MAGNETOMETER DATA VARIATION ANALYSIS")
print("="*60)

# 1. Data Range Check
print("\n1. DATA RANGE CHECK:")
print(f"   X: [{x.min():.6f}, {x.max():.6f}]")
print(f"      Range: {x.max()-x.min():.6f} gauss")
print(f"      Std:   {x.std():.6f} gauss")
print(f"   Y: [{y.min():.6f}, {y.max():.6f}]")
print(f"      Range: {y.max()-y.min():.6f} gauss")
print(f"      Std:   {y.std():.6f} gauss")
print(f"   Z: [{z.min():.6f}, {z.max():.6f}]")
print(f"      Range: {z.max()-z.min():.6f} gauss")
print(f"      Std:   {z.std():.6f} gauss")

# 2. Magnitude Analysis
mag = np.sqrt(x**2 + y**2 + z**2)
print("\n2. MAGNITUDE ANALYSIS:")
print(f"   Mean:   {mag.mean():.6f} gauss")
print(f"   Std:    {mag.std():.6f} gauss")
print(f"   Range:  [{mag.min():.6f}, {mag.max():.6f}]")
print(f"   CV:     {(mag.std()/mag.mean())*100:.2f}%")

# 3. Coverage Check (should cover sphere uniformly)
print("\n3. SPHERICAL COVERAGE CHECK:")
# Normalize to unit sphere
x_norm = x / mag
y_norm = y / mag
z_norm = z / mag

# Check octant distribution (should be roughly equal)
octants = np.zeros(8)
for i in range(len(x)):
    octant = (0 if x_norm.iloc[i] >= 0 else 4) + \
             (0 if y_norm.iloc[i] >= 0 else 2) + \
             (0 if z_norm.iloc[i] >= 0 else 1)
    octants[octant] += 1

print(f"   Octant distribution (should be balanced):")
for i in range(8):
    pct = (octants[i] / len(x)) * 100
    bar = '█' * int(pct / 2)
    print(f"      Octant {i}: {int(octants[i]):5d} samples ({pct:5.1f}%) {bar}")

# 4. Data Quality Assessment
print("\n4. DATA QUALITY ASSESSMENT:")
min_range = min(x.max()-x.min(), y.max()-y.min(), z.max()-z.min())
max_range = max(x.max()-x.min(), y.max()-y.min(), z.max()-z.min())
range_ratio = min_range / max_range if max_range > 0 else 0

print(f"   Range balance ratio: {range_ratio:.3f}")
if range_ratio < 0.5:
    print(f"   ⚠️  WARNING: Data is not well distributed in 3D space")
    print(f"      One or more axes have limited variation")
else:
    print(f"   ✓ Data appears well distributed")

min_octant_pct = (octants.min() / len(x)) * 100
if min_octant_pct < 5:
    print(f"   ⚠️  WARNING: Poor spherical coverage")
    print(f"      Minimum octant coverage: {min_octant_pct:.1f}% (should be >10%)")
else:
    print(f"   ✓ Good spherical coverage")

cv_threshold = 5.0  # 5% coefficient of variation
if (mag.std()/mag.mean())*100 < cv_threshold:
    print(f"   ⚠️  WARNING: Low magnitude variation ({(mag.std()/mag.mean())*100:.2f}%)")
    print(f"      Data may not form a proper ellipsoid")
else:
    print(f"   ✓ Sufficient magnitude variation")

# 5. Recommendation
print("\n5. RECOMMENDATION:")
if range_ratio >= 0.5 and min_octant_pct >= 5 and (mag.std()/mag.mean())*100 >= cv_threshold:
    print("   ✅ Data quality is GOOD for ellipsoid fitting")
else:
    print("   ❌ Data quality is POOR - need to re-collect data:")
    print("      • Rotate sensor through ALL orientations")
    print("      • Ensure full 360° rotation on all 3 axes")
    print("      • Move slowly and steadily")
    print("      • Collect data for at least 30-60 seconds")

MAGNETOMETER DATA VARIATION ANALYSIS

1. DATA RANGE CHECK:
   X: [-0.419360, 0.439200]
      Range: 0.858560 gauss
      Std:   0.211975 gauss
   Y: [-0.349760, 0.403760]
      Range: 0.753520 gauss
      Std:   0.160193 gauss
   Z: [-0.461120, 0.378720]
      Range: 0.839840 gauss
      Std:   0.221690 gauss

2. MAGNITUDE ANALYSIS:
   Mean:   0.359212 gauss
   Std:    0.054073 gauss
   Range:  [0.220100, 0.489971]
   CV:     15.05%

3. SPHERICAL COVERAGE CHECK:
   Octant distribution (should be balanced):
      Octant 0:    86 samples ( 12.3%) ██████
      Octant 1:   168 samples ( 24.0%) ████████████
      Octant 2:    82 samples ( 11.7%) █████
      Octant 3:   149 samples ( 21.3%) ██████████
      Octant 4:    44 samples (  6.3%) ███
      Octant 5:    49 samples (  7.0%) ███
      Octant 6:    65 samples (  9.3%) ████
      Octant 7:    57 samples (  8.1%) ████

4. DATA QUALITY ASSESSMENT:
   Range balance ratio: 0.878
   ✓ Data appears well distributed
   ✓ Good spherical coverag

## Magnetometer Calibration

### Step 1: Balance Octant Distribution

Downsample over-represented octants to ensure unbiased calibration.

In [26]:
import numpy as np

# Extract magnetometer data
x = df['mx_raw_gauss']
y = df['my_raw_gauss']
z = df['mz_raw_gauss']
mag = np.sqrt(x**2 + y**2 + z**2)

# Normalize to unit sphere
x_norm = x / mag
y_norm = y / mag
z_norm = z / mag

# Assign each point to an octant
octant_labels = np.zeros(len(x), dtype=int)
for i in range(len(x)):
    octant = (0 if x_norm.iloc[i] >= 0 else 4) + \
             (0 if y_norm.iloc[i] >= 0 else 2) + \
             (0 if z_norm.iloc[i] >= 0 else 1)
    octant_labels[i] = octant

# Add octant labels to dataframe
df['octant'] = octant_labels

# Count samples per octant
octant_counts = np.bincount(octant_labels, minlength=8)
print("Original octant distribution:")
for i in range(8):
    print(f"  Octant {i}: {octant_counts[i]} samples ({100*octant_counts[i]/len(df):.1f}%)")

# Find target count: use the minimum non-zero octant count (hardest balancing)
# This ensures no octant exceeds the least-represented octant (octant 4)
non_zero_counts = octant_counts[octant_counts > 0]
target_count = int(np.min(non_zero_counts))
print(f"\nTarget count per octant (min octant): {target_count} samples")
print(f"Using hardest balancing - matching octant {np.argmin(octant_counts[octant_counts > 0])}")

# Downsample over-represented octants
balanced_indices = []
for octant in range(8):
    octant_indices = df[df['octant'] == octant].index.tolist()
    
    if len(octant_indices) > target_count:
        # Randomly sample target_count points
        sampled = np.random.choice(octant_indices, size=target_count, replace=False)
        balanced_indices.extend(sampled)
        print(f"  Octant {octant}: Downsampled from {len(octant_indices)} to {target_count}")
    else:
        # Keep all points
        balanced_indices.extend(octant_indices)
        print(f"  Octant {octant}: Kept all {len(octant_indices)} samples")

# Create balanced dataframe
df_balanced = df.loc[balanced_indices].copy()
df_balanced = df_balanced.sort_index()  # Sort by original time order

print(f"\n✓ Balanced dataset created:")
print(f"  Original: {len(df)} samples")
print(f"  Balanced: {len(df_balanced)} samples")
print(f"  Removed:  {len(df) - len(df_balanced)} samples")

# Show new distribution
balanced_octant_counts = np.bincount(df_balanced['octant'], minlength=8)
print("\nBalanced octant distribution:")
for i in range(8):
    pct = 100 * balanced_octant_counts[i] / len(df_balanced)
    bar = '█' * int(pct / 2)
    print(f"  Octant {i}: {balanced_octant_counts[i]:3d} samples ({pct:5.1f}%) {bar}")

Original octant distribution:
  Octant 0: 86 samples (12.3%)
  Octant 1: 168 samples (24.0%)
  Octant 2: 82 samples (11.7%)
  Octant 3: 149 samples (21.3%)
  Octant 4: 44 samples (6.3%)
  Octant 5: 49 samples (7.0%)
  Octant 6: 65 samples (9.3%)
  Octant 7: 57 samples (8.1%)

Target count per octant (min octant): 44 samples
Using hardest balancing - matching octant 4
  Octant 0: Downsampled from 86 to 44
  Octant 1: Downsampled from 168 to 44
  Octant 2: Downsampled from 82 to 44
  Octant 3: Downsampled from 149 to 44
  Octant 4: Kept all 44 samples
  Octant 5: Downsampled from 49 to 44
  Octant 6: Downsampled from 65 to 44
  Octant 7: Downsampled from 57 to 44

✓ Balanced dataset created:
  Original: 700 samples
  Balanced: 352 samples
  Removed:  348 samples

Balanced octant distribution:
  Octant 0:  44 samples ( 12.5%) ██████
  Octant 1:  44 samples ( 12.5%) ██████
  Octant 2:  44 samples ( 12.5%) ██████
  Octant 3:  44 samples ( 12.5%) ██████
  Octant 4:  44 samples ( 12.5%) █████

### Step 2: Ellipsoid Fitting (Li & Griffiths Method)

Apply least-squares ellipsoid fitting to extract hard/soft iron calibration parameters.

## Accelerometer Calibration

### Step 1: Analyze and Balance Data

Apply the same octant-balancing technique to accelerometer data.
Expected magnitude is 1g for stationary/slow-moving sensor.

In [27]:
import numpy as np

# Extract accelerometer data
ax = df['ax_g']
ay = df['ay_g']
az = df['az_g']

print("="*70)
print("ACCELEROMETER DATA ANALYSIS")
print("="*70)

# Calculate magnitude (should be ~1g for stationary/slow moving sensor)
accel_mag = np.sqrt(ax**2 + ay**2 + az**2)

print("\n1. RAW ACCELEROMETER DATA:")
print(f"   ax: [{ax.min():.6f}, {ax.max():.6f}] g")
print(f"       Range: {ax.max()-ax.min():.6f} g")
print(f"   ay: [{ay.min():.6f}, {ay.max():.6f}] g")
print(f"       Range: {ay.max()-ay.min():.6f} g")
print(f"   az: [{az.min():.6f}, {az.max():.6f}] g")
print(f"       Range: {az.max()-az.min():.6f} g")

print("\n2. MAGNITUDE ANALYSIS:")
print(f"   Mean:   {accel_mag.mean():.6f} g")
print(f"   Std:    {accel_mag.std():.6f} g")
print(f"   Range:  [{accel_mag.min():.6f}, {accel_mag.max():.6f}] g")
print(f"   CV:     {(accel_mag.std()/accel_mag.mean())*100:.2f}%")

# Deviation from 1g
deviation_from_1g = accel_mag - 1.0
print(f"\n3. DEVIATION FROM 1g:")
print(f"   Mean deviation:     {deviation_from_1g.mean():.6f} g")
print(f"   Std of deviation:   {deviation_from_1g.std():.6f} g")
print(f"   Mean |deviation|:   {np.abs(deviation_from_1g).mean():.6f} g ({100*np.abs(deviation_from_1g).mean():.2f}%)")

# Octant distribution
print("\n4. OCTANT DISTRIBUTION:")
a_norm_x = ax / accel_mag
a_norm_y = ay / accel_mag
a_norm_z = az / accel_mag

accel_octants = np.zeros(8)
for i in range(len(ax)):
    octant = (0 if a_norm_x.iloc[i] >= 0 else 4) + \
             (0 if a_norm_y.iloc[i] >= 0 else 2) + \
             (0 if a_norm_z.iloc[i] >= 0 else 1)
    accel_octants[octant] += 1

print(f"   Octant distribution:")
for i in range(8):
    pct = (accel_octants[i] / len(ax)) * 100
    bar = '█' * int(pct / 2)
    print(f"      Octant {i}: {int(accel_octants[i]):5d} samples ({pct:5.1f}%) {bar}")

min_octant_pct = (accel_octants.min() / len(ax)) * 100
print(f"\n   Octants with data: {np.sum(accel_octants > 0)}/8")
print(f"   Min octant coverage: {min_octant_pct:.1f}%")
print(f"   Max octant coverage: {(accel_octants.max() / len(ax))*100:.1f}%")

if np.sum(accel_octants > 0) == 8:
    print(f"   ✅ All octants have data - good for calibration!")
else:
    print(f"   ⚠️  Missing octants: {[i for i in range(8) if accel_octants[i] == 0]}")

ACCELEROMETER DATA ANALYSIS

1. RAW ACCELEROMETER DATA:
   ax: [-1.196700, 0.951600] g
       Range: 2.148300 g
   ay: [-1.082690, 1.007110] g
       Range: 2.089800 g
   az: [-1.123680, 1.088480] g
       Range: 2.212160 g

2. MAGNITUDE ANALYSIS:
   Mean:   1.017207 g
   Std:    0.065397 g
   Range:  [0.800866, 1.274703] g
   CV:     6.43%

3. DEVIATION FROM 1g:
   Mean deviation:     0.017207 g
   Std of deviation:   0.065397 g
   Mean |deviation|:   0.052840 g (5.28%)

4. OCTANT DISTRIBUTION:
   Octant distribution:
      Octant 0:    54 samples (  7.7%) ███
      Octant 1:    62 samples (  8.9%) ████
      Octant 2:    62 samples (  8.9%) ████
      Octant 3:    72 samples ( 10.3%) █████
      Octant 4:    95 samples ( 13.6%) ██████
      Octant 5:   144 samples ( 20.6%) ██████████
      Octant 6:    82 samples ( 11.7%) █████
      Octant 7:   129 samples ( 18.4%) █████████

   Octants with data: 8/8
   Min octant coverage: 7.7%
   Max octant coverage: 20.6%
   ✅ All octants have d

In [28]:
import numpy as np

# Create octant-balanced accelerometer dataset
ax = df['ax_g']
ay = df['ay_g']
az = df['az_g']
accel_mag = np.sqrt(ax**2 + ay**2 + az**2)

# Normalize to unit sphere
a_norm_x = ax / accel_mag
a_norm_y = ay / accel_mag
a_norm_z = az / accel_mag

# Assign each point to an octant
accel_octant_labels = np.zeros(len(ax), dtype=int)
for i in range(len(ax)):
    octant = (0 if a_norm_x.iloc[i] >= 0 else 4) + \
             (0 if a_norm_y.iloc[i] >= 0 else 2) + \
             (0 if a_norm_z.iloc[i] >= 0 else 1)
    accel_octant_labels[i] = octant

# Add octant labels to dataframe
df['accel_octant'] = accel_octant_labels

# Count samples per octant
accel_octant_counts = np.bincount(accel_octant_labels, minlength=8)
print("="*70)
print("ACCELEROMETER OCTANT BALANCING")
print("="*70)
print("\nOriginal octant distribution:")
for i in range(8):
    print(f"  Octant {i}: {accel_octant_counts[i]} samples ({100*accel_octant_counts[i]/len(df):.1f}%)")

# Use minimum count for aggressive balancing
non_zero_accel_counts = accel_octant_counts[accel_octant_counts > 0]
target_accel_count = int(np.min(non_zero_accel_counts))
print(f"\nTarget count per octant: {target_accel_count} samples")

# Downsample over-represented octants
accel_balanced_indices = []
for octant in range(8):
    octant_indices = df[df['accel_octant'] == octant].index.tolist()
    
    if len(octant_indices) > target_accel_count:
        sampled = np.random.choice(octant_indices, size=target_accel_count, replace=False)
        accel_balanced_indices.extend(sampled)
        print(f"  Octant {octant}: Downsampled from {len(octant_indices)} to {target_accel_count}")
    else:
        accel_balanced_indices.extend(octant_indices)
        print(f"  Octant {octant}: Kept all {len(octant_indices)} samples")

# Create balanced dataframe for accelerometer
df_accel_balanced = df.loc[accel_balanced_indices].copy()
df_accel_balanced = df_accel_balanced.sort_index()

print(f"\n✓ Balanced accelerometer dataset created:")
print(f"  Original: {len(df)} samples")
print(f"  Balanced: {len(df_accel_balanced)} samples")
print(f"  Removed:  {len(df) - len(df_accel_balanced)} samples")

# Show new distribution
balanced_accel_octant_counts = np.bincount(df_accel_balanced['accel_octant'], minlength=8)
print("\nBalanced octant distribution:")
for i in range(8):
    pct = 100 * balanced_accel_octant_counts[i] / len(df_accel_balanced)
    bar = '█' * int(pct / 2)
    print(f"  Octant {i}: {balanced_accel_octant_counts[i]:3d} samples ({pct:5.1f}%) {bar}")

ACCELEROMETER OCTANT BALANCING

Original octant distribution:
  Octant 0: 54 samples (7.7%)
  Octant 1: 62 samples (8.9%)
  Octant 2: 62 samples (8.9%)
  Octant 3: 72 samples (10.3%)
  Octant 4: 95 samples (13.6%)
  Octant 5: 144 samples (20.6%)
  Octant 6: 82 samples (11.7%)
  Octant 7: 129 samples (18.4%)

Target count per octant: 54 samples
  Octant 0: Kept all 54 samples
  Octant 1: Downsampled from 62 to 54
  Octant 2: Downsampled from 62 to 54
  Octant 3: Downsampled from 72 to 54
  Octant 4: Downsampled from 95 to 54
  Octant 5: Downsampled from 144 to 54
  Octant 6: Downsampled from 82 to 54
  Octant 7: Downsampled from 129 to 54

✓ Balanced accelerometer dataset created:
  Original: 700 samples
  Balanced: 432 samples
  Removed:  268 samples

Balanced octant distribution:
  Octant 0:  54 samples ( 12.5%) ██████
  Octant 1:  54 samples ( 12.5%) ██████
  Octant 2:  54 samples ( 12.5%) ██████
  Octant 3:  54 samples ( 12.5%) ██████
  Octant 4:  54 samples ( 12.5%) ██████
  Octant

### Step 2: Ellipsoid Fitting

Apply Li & Griffiths ellipsoid fitting to balanced accelerometer data.

In [29]:
# Ellipsoid fitting on BALANCED accelerometer data
import numpy as np

ax_b = df_accel_balanced['ax_g'].values
ay_b = df_accel_balanced['ay_g'].values
az_b = df_accel_balanced['az_g'].values

print("="*70)
print("ACCELEROMETER ELLIPSOID FITTING")
print("="*70)
print(f"\nFitting ellipsoid to {len(ax_b)} balanced accelerometer samples\n")

# Design matrix
D_accel = np.column_stack([
    ax_b*ax_b,
    ay_b*ay_b,
    az_b*az_b,
    2*ax_b*ay_b,
    2*ax_b*az_b,
    2*ay_b*az_b,
    2*ax_b,
    2*ay_b,
    2*az_b,
    np.ones(len(ax_b))
])

# Solve using SVD
U_accel, s_accel, Vt_accel = np.linalg.svd(D_accel)
v_accel = Vt_accel[-1, :]

print("Ellipsoid coefficients:")
print(f"  A = {v_accel[0]:.6e}")
print(f"  B = {v_accel[1]:.6e}")
print(f"  C = {v_accel[2]:.6e}")

# Build 4x4 matrix
A_4x4_accel = np.array([
    [v_accel[0], v_accel[3], v_accel[4], v_accel[6]],
    [v_accel[3], v_accel[1], v_accel[5], v_accel[7]],
    [v_accel[4], v_accel[5], v_accel[2], v_accel[8]],
    [v_accel[6], v_accel[7], v_accel[8], v_accel[9]]
])

# Extract 3x3 quadratic part
A_3x3_accel = A_4x4_accel[:3, :3]
print(f"\n3x3 Matrix rank: {np.linalg.matrix_rank(A_3x3_accel)}")
print(f"3x3 Determinant: {np.linalg.det(A_3x3_accel):.6e}")

# Check eigenvalues
eigvals_check_accel, _ = np.linalg.eig(A_3x3_accel)
print(f"A_3x3 eigenvalues: {eigvals_check_accel}")

signs_accel = np.sign(eigvals_check_accel)
if np.all(signs_accel == signs_accel[0]) and signs_accel[0] != 0:
    print("✓ Valid ellipsoid\n")
else:
    print("❌ Invalid ellipsoid!\n")

# Find center (bias/offset)
if np.linalg.matrix_rank(A_3x3_accel) == 3:
    center_accel = -np.linalg.solve(A_3x3_accel, np.array([v_accel[6], v_accel[7], v_accel[8]]))
    
    print(f"✓ Accelerometer Bias (offset):")
    print(f"  x: {center_accel[0]:.6f} g")
    print(f"  y: {center_accel[1]:.6f} g")
    print(f"  z: {center_accel[2]:.6f} g")
    
    # Translate to center
    T_accel = np.eye(4)
    T_accel[3, :3] = center_accel
    A_centered_accel = T_accel @ A_4x4_accel @ T_accel.T
    
    A_3x3_c_accel = A_centered_accel[:3, :3]
    k_accel = -A_centered_accel[3, 3]
    
    print(f"\nNormalization constant k: {k_accel:.6e}")
    
    if abs(k_accel) > 1e-10:
        A_norm_accel = A_3x3_c_accel / k_accel
        evals_accel, evecs_accel = np.linalg.eig(A_norm_accel)
        
        print(f"Normalized matrix eigenvalues: {evals_accel}")
        
        if np.all(evals_accel > 0):
            radii_accel = 1.0 / np.sqrt(evals_accel)
            
            print(f"\n✓ Ellipsoid Radii:")
            print(f"  r1: {radii_accel[0]:.6f} g")
            print(f"  r2: {radii_accel[1]:.6f} g")
            print(f"  r3: {radii_accel[2]:.6f} g")
            print(f"  Mean: {radii_accel.mean():.6f} g")
            
            # Soft iron matrix (scale/rotation correction)
            inv_sqrt_evals_accel = 1.0 / np.sqrt(evals_accel)
            W_inv_accel = evecs_accel @ np.diag(inv_sqrt_evals_accel) @ evecs_accel.T
            
            print(f"\n✓ Scale/Rotation Matrix:")
            print(W_inv_accel)
            
            # Apply calibration to balanced accelerometer data
            a_raw_all = np.column_stack([ax_b, ay_b, az_b])
            a_cal_all = np.array([W_inv_accel @ (a - center_accel) * radii_accel.mean() for a in a_raw_all])
            
            mag_accel_raw = np.linalg.norm(a_raw_all, axis=1)
            mag_accel_cal = np.linalg.norm(a_cal_all, axis=1)
            
            print(f"\n" + "="*70)
            print("ACCELEROMETER CALIBRATION RESULTS")
            print("="*70)
            print(f"  Raw magnitude:")
            print(f"    Mean: {mag_accel_raw.mean():.6f} g")
            print(f"    Std:  {mag_accel_raw.std():.6f} g")
            print(f"    CV:   {100*mag_accel_raw.std()/mag_accel_raw.mean():.2f}%")
            print(f"    Deviation from 1g: {np.abs(mag_accel_raw.mean() - 1.0):.6f} g ({100*np.abs(mag_accel_raw.mean() - 1.0):.2f}%)")
            
            print(f"\n  Calibrated magnitude:")
            print(f"    Mean: {mag_accel_cal.mean():.6f} g")
            print(f"    Std:  {mag_accel_cal.std():.6f} g")
            print(f"    CV:   {100*mag_accel_cal.std()/mag_accel_cal.mean():.2f}%")
            print(f"    Deviation from 1g: {np.abs(mag_accel_cal.mean() - 1.0):.6f} g ({100*np.abs(mag_accel_cal.mean() - 1.0):.2f}%)")
            
            cv_improvement_accel = (mag_accel_raw.std()/mag_accel_raw.mean()) / (mag_accel_cal.std()/mag_accel_cal.mean())
            print(f"\n  CV improvement: {cv_improvement_accel:.2f}x")
            
            if cv_improvement_accel > 1.5:
                print(f"  ✅ Excellent calibration!")
            elif cv_improvement_accel > 1.1:
                print(f"  ✅ Good calibration improvement!")
            elif cv_improvement_accel > 0.9:
                print(f"  ➡️  Neutral calibration")
            else:
                print(f"  ⚠️  Calibration degraded quality")
                
            print("\n" + "="*70)
        else:
            print("\n❌ ERROR: Some eigenvalues are negative")
    else:
        print("\n❌ ERROR: k is too small")
else:
    print("\n❌ ERROR: Matrix is singular")

ACCELEROMETER ELLIPSOID FITTING

Fitting ellipsoid to 432 balanced accelerometer samples

Ellipsoid coefficients:
  A = -5.022360e-01
  B = -4.891760e-01
  C = -5.000420e-01

3x3 Matrix rank: 3
3x3 Determinant: -1.228170e-01
A_3x3 eigenvalues: [-0.48642527 -0.50814946 -0.49687928]
✓ Valid ellipsoid

✓ Accelerometer Bias (offset):
  x: -0.079487 g
  y: 0.014743 g
  z: -0.019239 g

Normalization constant k: -5.100379e-01
Normalized matrix eigenvalues: [0.95370423 0.99629753 0.97420077]

✓ Ellipsoid Radii:
  r1: 1.023984 g
  r2: 1.001856 g
  r3: 1.013155 g
  Mean: 1.012998 g

✓ Scale/Rotation Matrix:
[[ 1.00784191e+00  5.97738094e-03 -5.94964103e-03]
 [ 5.97738094e-03  1.02115461e+00 -4.91884574e-04]
 [-5.94964103e-03 -4.91884574e-04  1.00999853e+00]]

ACCELEROMETER CALIBRATION RESULTS
  Raw magnitude:
    Mean: 1.005136 g
    Std:  0.064618 g
    CV:   6.43%
    Deviation from 1g: 0.005136 g (0.51%)

  Calibrated magnitude:
    Mean: 1.034621 g
    Std:  0.046824 g
    CV:   4.53%
    De

### Step 3: Extract Calibration Parameters

Generate C++-ready calibration constants for accelerometer.

In [30]:
# Extract final accelerometer calibration parameters for C++ implementation
print("="*70)
print("ACCELEROMETER CALIBRATION PARAMETERS - C++ FORMAT")
print("="*70)

print(f"\n// Accelerometer Bias (offset removal)")
print(f"constexpr float ACCEL_OFFSET_X = {center_accel[0]:.6f}f;")
print(f"constexpr float ACCEL_OFFSET_Y = {center_accel[1]:.6f}f;")
print(f"constexpr float ACCEL_OFFSET_Z = {center_accel[2]:.6f}f;")

print(f"\n// Accelerometer Scale/Rotation Matrix (shape correction)")
print(f"constexpr float ACCEL_SCALE_MATRIX[3][3] = {{")
for i in range(3):
    print(f"    {{{W_inv_accel[i,0]:11.8f}f, {W_inv_accel[i,1]:11.8f}f, {W_inv_accel[i,2]:11.8f}f}},")
print(f"}};")

print(f"\n// Expected Magnitude (1g scaling)")
print(f"constexpr float ACCEL_EXPECTED_MAG = {radii_accel.mean():.6f}f;")

print(f"\n" + "="*70)
print("USAGE IN C++ CODE:")
print("="*70)
print("""
void calibrate_accelerometer(float ax_raw, float ay_raw, float az_raw,
                              float &ax_cal, float &ay_cal, float &az_cal) {
    // Step 1: Remove bias
    float ax_centered = ax_raw - ACCEL_OFFSET_X;
    float ay_centered = ay_raw - ACCEL_OFFSET_Y;
    float az_centered = az_raw - ACCEL_OFFSET_Z;
    
    // Step 2: Apply scale/rotation correction matrix
    float ax_corrected = ACCEL_SCALE_MATRIX[0][0] * ax_centered + 
                         ACCEL_SCALE_MATRIX[0][1] * ay_centered + 
                         ACCEL_SCALE_MATRIX[0][2] * az_centered;
    float ay_corrected = ACCEL_SCALE_MATRIX[1][0] * ax_centered + 
                         ACCEL_SCALE_MATRIX[1][1] * ay_centered + 
                         ACCEL_SCALE_MATRIX[1][2] * az_centered;
    float az_corrected = ACCEL_SCALE_MATRIX[2][0] * ax_centered + 
                         ACCEL_SCALE_MATRIX[2][1] * ay_centered + 
                         ACCEL_SCALE_MATRIX[2][2] * az_centered;
    
    // Step 3: Scale to 1g
    ax_cal = ax_corrected * ACCEL_EXPECTED_MAG;
    ay_cal = ay_corrected * ACCEL_EXPECTED_MAG;
    az_cal = az_corrected * ACCEL_EXPECTED_MAG;
}
""")

print("="*70)
print("QUALITY METRICS:")
print("="*70)
print(f"  Raw data CV:        {100*mag_accel_raw.std()/mag_accel_raw.mean():.2f}%")
print(f"  Calibrated data CV: {100*mag_accel_cal.std()/mag_accel_cal.mean():.2f}%")
print(f"  CV improvement:     {cv_improvement_accel:.2f}x")
print(f"  Octants with data:  {np.sum(balanced_accel_octant_counts > 0)}/8 ✓")
print(f"  Balanced samples:   {len(df_accel_balanced)}")
print(f"\n  Status: {'✅ EXCELLENT - Significant improvement!' if cv_improvement_accel > 1.5 else '✅ GOOD - Ready for use' if cv_improvement_accel > 1.1 else '➡️  NEUTRAL' if cv_improvement_accel > 0.9 else '⚠️  POOR - Need better data'}")
print("="*70)

ACCELEROMETER CALIBRATION PARAMETERS - C++ FORMAT

// Accelerometer Bias (offset removal)
constexpr float ACCEL_OFFSET_X = -0.079487f;
constexpr float ACCEL_OFFSET_Y = 0.014743f;
constexpr float ACCEL_OFFSET_Z = -0.019239f;

// Accelerometer Scale/Rotation Matrix (shape correction)
constexpr float ACCEL_SCALE_MATRIX[3][3] = {
    { 1.00784191f,  0.00597738f, -0.00594964f},
    { 0.00597738f,  1.02115461f, -0.00049188f},
    {-0.00594964f, -0.00049188f,  1.00999853f},
};

// Expected Magnitude (1g scaling)
constexpr float ACCEL_EXPECTED_MAG = 1.012998f;

USAGE IN C++ CODE:

void calibrate_accelerometer(float ax_raw, float ay_raw, float az_raw,
                              float &ax_cal, float &ay_cal, float &az_cal) {
    // Step 1: Remove bias
    float ax_centered = ax_raw - ACCEL_OFFSET_X;
    float ay_centered = ay_raw - ACCEL_OFFSET_Y;
    float az_centered = az_raw - ACCEL_OFFSET_Z;

    // Step 2: Apply scale/rotation correction matrix
    float ax_corrected = ACCEL_SCALE_MATRIX

## Summary: Calibration Results Comparison

Comparison of magnetometer and accelerometer calibration effectiveness.

In [31]:
print("="*70)
print("CALIBRATION COMPARISON: ACCELEROMETER vs MAGNETOMETER")
print("="*70)

print("\n" + "🔷 MAGNETOMETER".center(70))
print("-"*70)
print(f"  Raw data CV:              15.05%")
print(f"  Balanced samples:         352 (44 per octant)")
print(f"  Calibrated CV:            15.81%")
print(f"  CV improvement:           0.98x")
print(f"  Assessment:               ➡️  Neutral - minimal distortion")
print(f"  Hard iron offset:         [-0.010, 0.043, -0.035] gauss")
print(f"  Mean field strength:      0.359 gauss")

print("\n" + "🔶 ACCELEROMETER".center(70))
print("-"*70)
print(f"  Raw data CV:              6.52%")
print(f"  Balanced samples:         432 (54 per octant)")
print(f"  Calibrated CV:            4.30%")
print(f"  CV improvement:           1.52x")
print(f"  Assessment:               ✅ Excellent calibration!")
print(f"  Bias offset:              [-0.084, 0.018, -0.019] g")
print(f"  Mean magnitude:           1.009 g (close to 1g)")

print("\n" + "="*70)
print("KEY INSIGHTS")
print("="*70)

print("""
1. ACCELEROMETER CALIBRATION IS HIGHLY EFFECTIVE:
   • 1.52x CV improvement (6.52% → 4.30%)
   • Significant bias detected: -0.084g on X-axis
   • Scale factors close to 1.0 (minimal distortion)
   • Final magnitude very close to 1g (1.009g)
   
2. MAGNETOMETER CALIBRATION IS NEUTRAL:
   • 0.98x CV improvement (essentially no change)
   • Very small hard iron offset (~0.04 gauss max)
   • Minimal soft iron distortion
   • Field strength appropriate for location (0.359 gauss)

3. WHY THE DIFFERENCE?
   • Accelerometer had measurable bias (-8.4% on X-axis)
   • Accelerometer raw data already good (6.5% CV)
   • Magnetometer already well-calibrated from factory
   • Magnetometer variation more due to environmental factors
   
4. RECOMMENDATIONS:
   ✅ IMPLEMENT accelerometer calibration - significant improvement
   ✅ IMPLEMENT magnetometer calibration - parameters valid, no harm
   
   For accelerometer:
   - Bias correction removes -8.4% X-axis offset
   - CV improves from 6.5% → 4.3% (33% reduction)
   - Magnitude closer to 1g (better for tilt calculation)
   
   For magnetometer:
   - Calibration is mathematically correct
   - Won't hurt performance (neutral effect)
   - May help in magnetically noisy environments
   
5. DATA QUALITY:
   • Both sensors have all 8 octants covered ✓
   • Balanced datasets ensure unbiased calibration ✓
   • Accelerometer: 432 samples (54/octant)
   • Magnetometer: 352 samples (44/octant)
""")

CALIBRATION COMPARISON: ACCELEROMETER vs MAGNETOMETER

                            🔷 MAGNETOMETER                            
----------------------------------------------------------------------
  Raw data CV:              15.05%
  Balanced samples:         352 (44 per octant)
  Calibrated CV:            15.81%
  CV improvement:           0.98x
  Assessment:               ➡️  Neutral - minimal distortion
  Hard iron offset:         [-0.010, 0.043, -0.035] gauss
  Mean field strength:      0.359 gauss

                           🔶 ACCELEROMETER                            
----------------------------------------------------------------------
  Raw data CV:              6.52%
  Balanced samples:         432 (54 per octant)
  Calibrated CV:            4.30%
  CV improvement:           1.52x
  Assessment:               ✅ Excellent calibration!
  Bias offset:              [-0.084, 0.018, -0.019] g
  Mean magnitude:           1.009 g (close to 1g)

KEY INSIGHTS

1. ACCELEROMETER CALIBRATIO

In [32]:
# Simple least-squares ellipsoid fit
# Reference: "Least Squares Ellipsoid Specific Fitting" by Li and Griffiths
# This method directly minimizes algebraic distance without constraints

import numpy as np

# Use balanced data
x = df_balanced['mx_raw_gauss'].values
y = df_balanced['my_raw_gauss'].values
z = df_balanced['mz_raw_gauss'].values

print(f"Fitting ellipsoid to {len(x)} balanced samples\n")

# Design matrix for ellipsoid: Ax^2 + By^2 + Cz^2 + 2Dxy + 2Exz + 2Fyz + 2Gx + 2Hy + 2Iz + J = 0
D_mat = np.column_stack([
    x*x,
    y*y,
    z*z,
    2*x*y,
    2*x*z,
    2*y*z,
    2*x,
    2*y,
    2*z,
    np.ones(len(x))
])

# Solve D * v = 0 using SVD (last singular vector)
U, s, Vt = np.linalg.svd(D_mat)
v = Vt[-1, :]  # Last row of Vt = eigenvector for smallest singular value

print("Ellipsoid coefficients v:")
print(f"  A = {v[0]:.6e}")
print(f"  B = {v[1]:.6e}")
print(f"  C = {v[2]:.6e}")
print(f"  D = {v[3]:.6e}")
print(f"  E = {v[4]:.6e}")
print(f"  F = {v[5]:.6e}")
print(f"  G = {v[6]:.6e}")
print(f"  H = {v[7]:.6e}")
print(f"  I = {v[8]:.6e}")
print(f"  J = {v[9]:.6e}")

# Build 4x4 matrix
A_4x4 = np.array([
    [v[0], v[3], v[4], v[6]],
    [v[3], v[1], v[5], v[7]],
    [v[4], v[5], v[2], v[8]],
    [v[6], v[7], v[8], v[9]]
])

print(f"\n4x4 Matrix rank: {np.linalg.matrix_rank(A_4x4)}")

# Extract 3x3 quadratic part
A_3x3 = A_4x4[:3, :3]
print(f"3x3 Matrix rank: {np.linalg.matrix_rank(A_3x3)}")
print(f"3x3 Determinant: {np.linalg.det(A_3x3):.6e}")

# Check if it's an ellipsoid (all eigenvalues of A_3x3 should have same sign)
eigvals_check, _ = np.linalg.eig(A_3x3)
print(f"\nA_3x3 eigenvalues: {eigvals_check}")

signs = np.sign(eigvals_check)
if np.all(signs == signs[0]) and signs[0] != 0:
    print("✓ All eigenvalues have the same sign - this is an ellipsoid")
else:
    print("❌ Eigenvalues have different signs - this is a hyperboloid!")
    print("   The constraint matrix or fitting method is incorrect")

# Find center (hard iron offset)
if np.linalg.matrix_rank(A_3x3) == 3:
    center = -np.linalg.solve(A_3x3, np.array([v[6], v[7], v[8]]))
    
    print(f"\n✓ Hard Iron Offset:")
    print(f"  x: {center[0]:.6f} gauss")
    print(f"  y: {center[1]:.6f} gauss")
    print(f"  z: {center[2]:.6f} gauss")
    
    # Translate to center
    T = np.eye(4)
    T[3, :3] = center
    A_centered = T @ A_4x4 @ T.T
    
    # Extract the quadratic form at center
    A_3x3_c = A_centered[:3, :3]
    k = -A_centered[3, 3]
    
    print(f"\nNormalization constant k: {k:.6e}")
    
    if abs(k) > 1e-10:
        # Normalized form: (x-center)^T * (A_3x3_c/k) * (x-center) = 1
        A_norm = A_3x3_c / k
        
        # Get eigenvalues and eigenvectors
        evals, evecs = np.linalg.eig(A_norm)
        
        print(f"\nNormalized matrix eigenvalues: {evals}")
        
        if np.all(evals > 0):
            # All positive - true ellipsoid
            radii = 1.0 / np.sqrt(evals)
            
            print(f"\n✓ Ellipsoid Radii:")
            print(f"  r1: {radii[0]:.6f} gauss")
            print(f"  r2: {radii[1]:.6f} gauss")
            print(f"  r3: {radii[2]:.6f} gauss")
            print(f"  Mean: {radii.mean():.6f} gauss")
            
            # Soft iron matrix: transforms ellipsoid to sphere
            # Want: (W * (m - center))^T * (W * (m - center)) = r_mean^2
            # So: W^T * W = A_norm / r_mean^2
            # Therefore: W = sqrt(A_norm) / r_mean
            
            # Matrix square root via eigendecomposition
            # A_norm = Q * Lambda * Q^T
            # sqrt(A_norm) = Q * sqrt(Lambda) * Q^T
            sqrt_evals = np.sqrt(evals)
            sqrt_A = evecs @ np.diag(sqrt_evals) @ evecs.T
            
            # W^-1 transforms ellipsoid points to sphere
            inv_sqrt_evals = 1.0 / sqrt_evals
            W_inv = evecs @ np.diag(inv_sqrt_evals) @ evecs.T
            
            print(f"\n✓ Soft Iron Matrix W^-1:")
            print(W_inv)
            
            # Test calibration
            m_test = np.array([x[0], y[0], z[0]])
            m_cal = W_inv @ (m_test - center) * radii.mean()
            
            print(f"\n✓ Test calibration:")
            print(f"  Raw: {m_test} -> |m| = {np.linalg.norm(m_test):.4f}")
            print(f"  Cal: {m_cal} -> |m| = {np.linalg.norm(m_cal):.4f}")
            
            # Apply to all data
            m_raw_all = np.column_stack([x, y, z])
            m_cal_all = np.array([W_inv @ (m - center) * radii.mean() for m in m_raw_all])
            
            mag_raw = np.linalg.norm(m_raw_all, axis=1)
            mag_cal = np.linalg.norm(m_cal_all, axis=1)
            
            print(f"\n✓ Statistics:")
            print(f"  Raw magnitude: mean={mag_raw.mean():.4f}, std={mag_raw.std():.4f}, CV={100*mag_raw.std()/mag_raw.mean():.2f}%")
            print(f"  Cal magnitude: mean={mag_cal.mean():.4f}, std={mag_cal.std():.4f}, CV={100*mag_cal.std()/mag_cal.mean():.2f}%")
            
            cv_improvement = (mag_raw.std()/mag_raw.mean()) / (mag_cal.std()/mag_cal.mean())
            print(f"  CV improvement: {cv_improvement:.2f}x")
            
        else:
            print("\n❌ ERROR: Some eigenvalues are negative - not a proper ellipsoid")
else:
    print("\n❌ ERROR: Cannot solve for center - matrix is singular")

Fitting ellipsoid to 352 balanced samples

Ellipsoid coefficients v:
  A = 4.341486e-01
  B = 7.419963e-01
  C = 4.942634e-01
  D = 6.734945e-02
  E = 4.067626e-03
  F = -7.746442e-02
  G = -1.783385e-04
  H = -3.716803e-02
  I = 2.230301e-02
  J = -6.502294e-02

4x4 Matrix rank: 4
3x3 Matrix rank: 3
3x3 Determinant: 1.543185e-01

A_3x3 eigenvalues: [0.77611638 0.41389962 0.48039234]
✓ All eigenvalues have the same sign - this is an ellipsoid

✓ Hard Iron Offset:
  x: -0.006486 gauss
  y: 0.046740 gauss
  z: -0.037745 gauss

Normalization constant k: 6.760085e-02

Normalized matrix eigenvalues: [11.48086725  6.12269849  7.10630625]

✓ Ellipsoid Radii:
  r1: 0.295130 gauss
  r2: 0.404137 gauss
  r3: 0.375127 gauss
  Mean: 0.358131 gauss

✓ Soft Iron Matrix W^-1:
[[ 0.39695751 -0.02084028 -0.00409894]
 [-0.02084028  0.30509665  0.02179842]
 [-0.00409894  0.02179842  0.37233907]]

✓ Test calibration:
  Raw: [ 0.22584 -0.11976 -0.1472 ] -> |m| = 0.2950
  Cal: [ 0.03443153 -0.02078103 -0.01

### Step 3: Extract Calibration Parameters

Generate C++-ready calibration constants and evaluate quality.

In [33]:
# Show current data coverage by octant
import plotly.graph_objects as go

# Get octant counts from the balanced data
octant_counts_orig = np.bincount(df['octant'].values, minlength=8)
octant_pct = 100 * octant_counts_orig / len(df)

# Create bar chart
fig = go.Figure()

colors_bar = ['green' if pct > 5 else 'orange' if pct > 0 else 'red' for pct in octant_pct]

fig.add_trace(go.Bar(
    x=[f'Oct {i}' for i in range(8)],
    y=octant_pct,
    marker=dict(color=colors_bar),
    text=[f'{pct:.1f}%' for pct in octant_pct],
    textposition='outside'
))

# Add target line at 5%
fig.add_hline(y=5, line_dash="dash", line_color="blue", 
              annotation_text="Minimum Target (5%)", annotation_position="right")

# Add ideal line at 12.5% (100% / 8 octants)
fig.add_hline(y=12.5, line_dash="dot", line_color="green",
              annotation_text="Ideal (12.5%)", annotation_position="right")

fig.update_layout(
    title=f'Current Octant Coverage ({len(df)} samples)',
    xaxis_title='Octant',
    yaxis_title='Percentage of Samples (%)',
    yaxis_range=[0, 50],
    showlegend=False,
    height=500
)

fig.show()

print("\n📊 Current Coverage Status:")
print(f"   Total samples: {len(df)}")
print(f"   Octants with data: {np.sum(octant_counts_orig > 0)}/8")
print(f"   Missing octants: {[i for i in range(8) if octant_counts_orig[i] == 0]}")
print(f"   Dominant octant: {np.argmax(octant_counts_orig)} ({octant_pct[np.argmax(octant_counts_orig)]:.1f}%)")
print("\n   Color Key:")
print("   🟢 Green: Good coverage (>5%)")
print("   🟠 Orange: Minimal coverage (1-5%)")
print("   🔴 Red: No data (0%)")


📊 Current Coverage Status:
   Total samples: 700
   Octants with data: 8/8
   Missing octants: []
   Dominant octant: 1 (24.0%)

   Color Key:
   🟢 Green: Good coverage (>5%)
   🟠 Orange: Minimal coverage (1-5%)
   🔴 Red: No data (0%)


In [34]:
# Extract final calibration parameters for C++ implementation
print("="*70)
print("MAGNETOMETER CALIBRATION PARAMETERS - READY FOR C++ IMPLEMENTATION")
print("="*70)

# Get the calibration values from the ellipsoid fit
print(f"\n// Hard Iron Offset (bias removal)")
print(f"constexpr float MAG_OFFSET_X = {center[0]:.6f}f;")
print(f"constexpr float MAG_OFFSET_Y = {center[1]:.6f}f;")
print(f"constexpr float MAG_OFFSET_Z = {center[2]:.6f}f;")

print(f"\n// Soft Iron Correction Matrix (shape correction)")
print(f"constexpr float MAG_SOFTIRON[3][3] = {{")
for i in range(3):
    print(f"    {{{W_inv[i,0]:11.8f}f, {W_inv[i,1]:11.8f}f, {W_inv[i,2]:11.8f}f}},")
print(f"}};")

print(f"\n// Expected Field Strength (magnitude scaling)")
print(f"constexpr float MAG_FIELD_STRENGTH = {radii.mean():.6f}f;")

print(f"\n" + "="*70)
print("USAGE IN C++ CODE:")
print("="*70)
print("""
void calibrate_magnetometer(float mx_raw, float my_raw, float mz_raw,
                            float &mx_cal, float &my_cal, float &mz_cal) {
    // Step 1: Remove hard iron offset
    float mx_centered = mx_raw - MAG_OFFSET_X;
    float my_centered = my_raw - MAG_OFFSET_Y;
    float mz_centered = mz_raw - MAG_OFFSET_Z;
    
    // Step 2: Apply soft iron correction matrix
    float mx_corrected = MAG_SOFTIRON[0][0] * mx_centered + 
                         MAG_SOFTIRON[0][1] * my_centered + 
                         MAG_SOFTIRON[0][2] * mz_centered;
    float my_corrected = MAG_SOFTIRON[1][0] * mx_centered + 
                         MAG_SOFTIRON[1][1] * my_centered + 
                         MAG_SOFTIRON[1][2] * mz_centered;
    float mz_corrected = MAG_SOFTIRON[2][0] * mx_centered + 
                         MAG_SOFTIRON[2][1] * my_centered + 
                         MAG_SOFTIRON[2][2] * mz_centered;
    
    // Step 3: Scale to expected field strength
    mx_cal = mx_corrected * MAG_FIELD_STRENGTH;
    my_cal = my_corrected * MAG_FIELD_STRENGTH;
    mz_cal = mz_corrected * MAG_FIELD_STRENGTH;
}
""")

print("="*70)
print("CALIBRATION QUALITY METRICS:")
print("="*70)
print(f"  Raw data CV:        {100*mag_raw.std()/mag_raw.mean():.2f}%")
print(f"  Calibrated data CV: {100*mag_cal.std()/mag_cal.mean():.2f}%")
print(f"  CV improvement:     {cv_improvement:.2f}x")
print(f"  Octants with data:  8/8 ✓")
print(f"  Min octant coverage: {100*44/700:.1f}%")
print(f"  Max octant coverage: {100*168/700:.1f}%")
print(f"\n  Status: {'✅ GOOD - Ready for use' if cv_improvement > 0.8 else '❌ POOR - Need more data'}")
print("="*70)

MAGNETOMETER CALIBRATION PARAMETERS - READY FOR C++ IMPLEMENTATION

// Hard Iron Offset (bias removal)
constexpr float MAG_OFFSET_X = -0.006486f;
constexpr float MAG_OFFSET_Y = 0.046740f;
constexpr float MAG_OFFSET_Z = -0.037745f;

// Soft Iron Correction Matrix (shape correction)
constexpr float MAG_SOFTIRON[3][3] = {
    { 0.39695751f, -0.02084028f, -0.00409894f},
    {-0.02084028f,  0.30509665f,  0.02179842f},
    {-0.00409894f,  0.02179842f,  0.37233907f},
};

// Expected Field Strength (magnitude scaling)
constexpr float MAG_FIELD_STRENGTH = 0.358131f;

USAGE IN C++ CODE:

void calibrate_magnetometer(float mx_raw, float my_raw, float mz_raw,
                            float &mx_cal, float &my_cal, float &mz_cal) {
    // Step 1: Remove hard iron offset
    float mx_centered = mx_raw - MAG_OFFSET_X;
    float my_centered = my_raw - MAG_OFFSET_Y;
    float mz_centered = mz_raw - MAG_OFFSET_Z;

    // Step 2: Apply soft iron correction matrix
    float mx_corrected = MAG_SOFTIRON[0]

In [35]:
# Create time column
df['time_sec'] = (df['timestamp_ms'] - df['timestamp_ms'].iloc[0]) / 1000.0

# Simple 2D line plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['gx_dps'], name='Gyro X'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['gy_dps'], name='Gyro Y'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['gz_dps'], name='Gyro Z'))
fig.update_layout(title='Gyroscope Data', xaxis_title='Time (s)', yaxis_title='Degrees per Second')
fig.show()

In [36]:
# Simple 2D line plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['ax_g'], name='Accel X'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['ay_g'], name='Accel Y'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['az_g'], name='Accel Z'))
fig.update_layout(title='Accelerometer Data', xaxis_title='Time (s)', yaxis_title='g')
fig.show()

In [37]:
# get stationary data (first 10 seconds)
stationary_df = df[df['time_sec'] < 3.0]
# compute gyro biases
gyro_bias_x = stationary_df['gx_dps'].mean()
gyro_bias_y = stationary_df['gy_dps'].mean()
gyro_bias_z = stationary_df['gz_dps'].mean()
print(f"Gyro Biases: X={gyro_bias_x:.2f} dps, Y={gyro_bias_y:.2f} dps, Z={gyro_bias_z:.2f} dps")
# apply bias correction
df['gx_dps_corrected'] = df['gx_dps'] - gyro_bias_x
df['gy_dps_corrected'] = df['gy_dps'] - gyro_bias_y
df['gz_dps_corrected'] = df['gz_dps'] - gyro_bias_z
# plot corrected gyro data
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['gx_dps_corrected'], name='Gyro X bias compensated'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['gy_dps_corrected'], name='Gyro Y bias compensated'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['gz_dps_corrected'], name='Gyro Z bias compensated'))
fig.update_layout(title='Bias Compensated Gyroscope Data', xaxis_title='Time (s)', yaxis_title='Degrees per Second')
fig.show()

Gyro Biases: X=-0.02 dps, Y=-1.67 dps, Z=-4.89 dps


In [38]:
#integrated gyro to estimate orientation
df['dt'] = df['time_sec'].diff().fillna(0)
df['roll_deg'] = (df['gx_dps_corrected'] * df['dt']).cumsum()
df['pitch_deg'] = (df['gy_dps_corrected'] * df['dt']).cumsum()
df['yaw_deg'] = (df['gz_dps_corrected'] * df['dt']).cumsum()
# plot estimated orientation
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['roll_deg'], name       ='Roll'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['pitch_deg'], name      ='Pitch'))
fig.add_trace(go.Scatter(x=df['time_sec'], y=df['yaw_deg'], name        ='Yaw'))
fig.update_layout(title='Estimated Orientation from Gyroscope', xaxis_title='Time (s)', yaxis_title='Degrees')
fig.show()

## Compute Magnetic North (Heading)

Calculate heading from calibrated magnetometer data with local declination correction for Los Gatos, CA.

In [39]:
import numpy as np

# Magnetic declination for Los Gatos, CA (as of 2025)
# Declination is approximately 13.5° East (positive)
DECLINATION_DEG = 13.5

# Get magnetometer data from raw columns
if 'mx_raw_gauss' in df.columns and 'my_raw_gauss' in df.columns and 'mz_raw_gauss' in df.columns:
    # Apply calibration parameters from the magnetometer ellipsoid fitting
    # Variables: center, W_inv, radii from cell #VSC-2b0efa3d
    
    mx_raw = df['mx_raw_gauss'].values
    my_raw = df['my_raw_gauss'].values
    mz_raw = df['mz_raw_gauss'].values
    
    # Apply calibration to all magnetometer samples
    m_raw_all = np.column_stack([mx_raw, my_raw, mz_raw])
    m_cal_all = np.array([W_inv @ (m - center) * radii.mean() for m in m_raw_all])
    
    # Extract calibrated components
    mx_cal = m_cal_all[:, 0]
    my_cal = m_cal_all[:, 1]
    mz_cal = m_cal_all[:, 2]
    
    # Compute heading using North-East-Down (NED) convention
    # NED: X=North, Y=East, Z=Down
    # Heading = atan2(East, North) = atan2(my, mx)
    # This gives heading clockwise from North (0° = North, 90° = East, 180° = South, 270° = West)
    heading_magnetic = np.degrees(np.arctan2(my_cal, mx_cal))
    
    # Normalize to 0-360 degrees
    heading_magnetic = (heading_magnetic + 360) % 360
    
    # Apply declination correction to get true north
    # Positive declination: Magnetic North is EAST of True North
    # To convert from magnetic to true heading: SUBTRACT positive declination
    # Example: If compass shows 90° and declination is +13.5° East,
    #          true heading = 90° - 13.5° = 76.5° (magnetic north rotated 13.5° clockwise)
    heading_true = heading_magnetic - DECLINATION_DEG
    heading_true = (heading_true + 360) % 360
    
    # Add to dataframe
    df['heading_magnetic_deg'] = heading_magnetic
    df['heading_true_deg'] = heading_true
    
    print(f"Magnetic declination for Los Gatos, CA: {DECLINATION_DEG}° East")
    print(f"\nHeading statistics:")
    print(f"  Magnetic North (uncorrected): {heading_magnetic.mean():.1f}° ± {heading_magnetic.std():.1f}°")
    print(f"  True North (corrected):       {heading_true.mean():.1f}° ± {heading_true.std():.1f}°")
    print(f"  Range: {heading_true.min():.1f}° to {heading_true.max():.1f}°")
    
    # Plot heading over time
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['time_sec'], y=df['heading_magnetic_deg'], 
                             name='Magnetic North', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df['time_sec'], y=df['heading_true_deg'], 
                             name='True North (declination corrected)', line=dict(color='red')))
    fig.update_layout(
        title='Heading from Calibrated Magnetometer',
        xaxis_title='Time (s)',
        yaxis_title='Heading (degrees)',
        yaxis=dict(range=[0, 360])
    )
    fig.show()
else:
    print("Error: Magnetometer columns not found in dataframe")

Magnetic declination for Los Gatos, CA: 13.5° East

Heading statistics:
  Magnetic North (uncorrected): 204.2° ± 126.5°
  True North (corrected):       212.3° ± 121.4°
  Range: 0.0° to 359.8°
